<a href="https://colab.research.google.com/github/MarT1AZ/Apple_DB/blob/main/Generating_script/GenData_AppleDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import hashlib
import random

# load data

* ### Please directly download form github to your local machine and then uplaod it to the colab for now, using wget cause error on read_csv()

# Prepare data for Account df

### Prepare hashed Password

In [54]:
digits = np.random.randint(0,10,size = (140,6))
hashed_passwords = []
for idx in range(len(digits)):
  pw = digits[idx]
  encoded_pw = str(pw[0]) + str(pw[1]) + str(pw[2]) + str(pw[3]) + str(pw[4]) + str(pw[5])
  hashed_passwords.append(hashlib.md5(encoded_pw.encode('ASCII')).hexdigest())
hashed_passwords[0:6]

['99dd3f4090d8fdc6500119f270ae60f6',
 '27268e6ac894c1ed4ca1d8921f5a69a0',
 'f3c448081aab09091f4f5ed1eddf1762',
 '33634c80f28bb8ed5190a3e4c006c170',
 'c04259ac5f289f13efa695db38f82149',
 'd33dc0346a9b4fefc8d567e4cc9ea689']

### Prepare address + country

* #### The data for USA states can be found here : https://github.com/kristw/gridmap-layout-thailand/blob/master/src/input/provinces.csv

* #### The data for thailand provinces can be found here : https://docs.google.com/spreadsheets/d/14wvnQygIX1eCVo7H5B7a96W1v5VCg6Q9yeRoESF6epw/edit?gid=0#gid=0

* #### The data for japan provinces can be found here : https://github.com/nobuf/list-of-cities-in-japan/blob/master/build/prefectures.csv

* #### the street address is generated with Chat GPT (I could find a way to mass generate the street address) see "street_address.csv"


In [6]:
#address include
# Street address
# zip code
# city/state
# phone

# USA_states_df = pd.read_csv('50 states and abbreviations - Sheet1.csv', on_bad_lines='skip')
# THAI_provinces_df = pd.read_csv('provinces.csv', on_bad_lines='skip')
# JAPAN_provinces_df = pd.read_csv('prefectures.csv', on_bad_lines='skip')
# street_df = pd.read_csv('street_address.csv', on_bad_lines='skip') # incldue zipcode

USA_states_df = pd.read_csv('50 states and abbreviations - Sheet1.csv')
THAI_provinces_df = pd.read_csv('provinces.csv')
JAPAN_provinces_df = pd.read_csv('prefectures.csv')
street_df = pd.read_csv('street_address.csv') # incldue zipcode

In [13]:
USA_states_df.shape

(51, 2)

In [14]:
street_df.shape

(92, 4)

In [55]:
# sampling 92 due to limit data
sample_USA_state = USA_states_df['State'].sample(30).values
sample_THAI_prov = THAI_provinces_df['enName'].sample(31).values
sample_JAPAN_prov = JAPAN_provinces_df['prefecture_en'].sample(31).values
# for JAPAN province, the city comes with 'city_name prefecture' so we need to remove it
sample_JAPAN_prov = [jp.split(' ')[0] for jp in sample_JAPAN_prov]
sample_city = [*sample_USA_state,*sample_THAI_prov,*sample_JAPAN_prov]
sample_street_address_indices = street_df.sample(92).index # only has 92 values
street_address = street_df.loc[sample_street_address_indices,'Address'].values
zip_code = street_df.loc[sample_street_address_indices,' ZIP'].values
final_address = []

# we know first 7 entries are USA and next 7 entries are THAI and last 6 are JAPAN
country = []
for i in range(0,30):
  country.append('USA')
for i in range(0,31):
  country.append('Thai')
for i in range(0,31):
  country.append('Japan')

for idx in range(0,92):
  final_address.append(street_address[idx] + ' ' + str(zip_code[idx]) + ' '+sample_city[idx])


# we have 92 entries but we need 8 more but to comabt unique constraint in the data base
# we instead sample 48 more making 140 entries in total

sample_USA_state = USA_states_df['State'].sample(18).values
sample_THAI_prov = THAI_provinces_df['enName'].sample(15).values
sample_JAPAN_prov = JAPAN_provinces_df['prefecture_en'].sample(15).values
sample_JAPAN_prov = [jp.split(' ')[0] for jp in sample_JAPAN_prov]

sample_city = [*sample_USA_state,*sample_THAI_prov,*sample_JAPAN_prov]

sample_street_address_indices = street_df.sample(48).index # only has 92 values
street_address = street_df.loc[sample_street_address_indices,'Address'].values
zip_code = street_df.loc[sample_street_address_indices,' ZIP'].values


for i in range(0,18):
  country.append('USA')
for i in range(0,15):
  country.append('Thai')
for i in range(0,15):
  country.append('Japan')

for idx in range(0,48):
  final_address.append(street_address[idx] + ' ' + str(zip_code[idx]) + ' '+sample_city[idx])
print(len(final_address))
final_address[:6]


140


['8779 Pine Rd 16501 Maryland',
 '4783 Hickory Ln 50613 Massachusetts',
 '4894 Maple Rd 64075 California',
 '3673 Cedar Blvd 27601 Vermont',
 '9990 Oak Dr 58102 Nebraska',
 '7558 Oak Rd 63108 District of Columbia']

In [56]:
len(country)

140

### Prepare names

In [57]:
fname_list = ['Eloise','Marie','Beatrice','Isla',
             'Ellen','Margaret','Maisie','Heidi',
              'Lily','Sara','Ben','Brandon','Justin','Steve',
             'Leonard','Muhammed','Declan','Ewan',
              'Hugo','Steven']

lname_list = ['Castaneda','Williams','Bull','Coles',
              'Whittaker','Rees','Peters','Brewer',
              'Goodwin','Russell','Rawlings','Singh',
              'Butler','Clark','Ward','Cross',
              'Connor','Carlson','Tate','Gilbert',]

# possible name combination is 400 names
all_possible_name = [(f + ' ' + l) for f in fname_list for l in lname_list]
all_possible_name[0:6]

['Eloise Castaneda',
 'Eloise Williams',
 'Eloise Bull',
 'Eloise Coles',
 'Eloise Whittaker',
 'Eloise Rees']

In [58]:
# sampling 25 names from 400 name
sample_names = []
name_indices = np.random.random_integers(0,399,140)
for ni in name_indices:
  sample_names.append(all_possible_name[ni])
sample_names[0:6]

<ipython-input-58-f06049553567>:3: DeprecationWarning: This function is deprecated. Please call randint(0, 399 + 1) instead
  name_indices = np.random.random_integers(0,399,140)


['Sara Russell',
 'Beatrice Peters',
 'Maisie Castaneda',
 'Beatrice Rawlings',
 'Steven Russell',
 'Marie Clark']

### prepare Email

In [59]:
# format fname.lname@gmail.com
def first_alph(text):
  if (len(text) < 4):
    text[0:len(text)]
  else:
    return text[0:4]

emails = []
for name in sample_names:
  fname,lname = name.split(' ')
  # using full fname
  # using 4 alphabet of lname
  emails.append(fname + '.' + first_alph(lname) + '@gmail.com')

### prepare phone number

In [60]:
def gen_phone():
  first = str(random.randint(100,999))
  second = str(random.randint(100,888))
  last = str(random.randint(1,9998))
  return f'{first}-{second}-{last}'

phone = []
for i in range(0,140):
  phone.append(gen_phone())
phone[:5]

['541-525-9113', '180-717-9888', '760-576-5640', '546-369-351', '499-614-727']

### prepare login_verification_method

In [61]:
login_verf_method = ['OTP','device','Email']
random_lvf = []
for i in range(0,140):
  random_lvf.append(login_verf_method[random.randint(0,2)])
random_lvf[:7]

['OTP', 'Email', 'device', 'device', 'Email', 'device', 'Email']

# Assembling data to Account df

In [66]:
Account_df = pd.DataFrame([],columns = ['Account_ID','Email','Phone_number',
                                        'Login_verification_method','country','name',
                                        'address','password'])

In [67]:
Account_df['Account_ID'] = pd.Series(list(range(10000,10140)))
Account_df['name'] = pd.Series(sample_names)
Account_df['address'] = pd.Series(final_address)
Account_df['password'] = pd.Series(hashed_passwords)
Account_df['country'] = pd.Series(country)
Account_df['Email'] = pd.Series(emails)
Account_df['Phone_number'] = pd.Series(phone)
Account_df['Login_verification_method'] = pd.Series(random_lvf)

In [68]:
Account_df.head()

,Account_ID,Email,Phone_number,Login_verification_method,country,name,address,password
0,10000,Sara.Russ@gmail.com,541-525-9113,OTP,USA,Sara Russell,8779 Pine Rd 16501 Maryland,99dd3f4090d8fdc6500119f270ae60f6
1,10001,Beatrice.Pete@gmail.com,180-717-9888,Email,USA,Beatrice Peters,4783 Hickory Ln 50613 Massachusetts,27268e6ac894c1ed4ca1d8921f5a69a0
2,10002,Maisie.Cast@gmail.com,760-576-5640,device,USA,Maisie Castaneda,4894 Maple Rd 64075 California,f3c448081aab09091f4f5ed1eddf1762
3,10003,Beatrice.Rawl@gmail.com,546-369-351,device,USA,Beatrice Rawlings,3673 Cedar Blvd 27601 Vermont,33634c80f28bb8ed5190a3e4c006c170
4,10004,Steven.Russ@gmail.com,499-614-727,Email,USA,Steven Russell,9990 Oak Dr 58102 Nebraska,c04259ac5f289f13efa695db38f82149


### check data intregity

In [69]:
Account_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Account_ID                 140 non-null    int64 
 1   Email                      140 non-null    object
 2   Phone_number               140 non-null    object
 3   Login_verification_method  140 non-null    object
 4   country                    140 non-null    object
 5   name                       140 non-null    object
 6   address                    140 non-null    object
 7   password                   140 non-null    object
dtypes: int64(1), object(7)
memory usage: 8.9+ KB


In [70]:
Account_df['Email'].nunique()

116

In [71]:
Account_df.to_csv('Apple_account.csv') # saving data

# Prepare data for apple physical store

### Prepare name + Address

In [72]:
# Apple Central World
# 4, 4/1-4/2, 4/4 ถนนราชดำริห์
# เขตปทุมวัน
# กรุงเทพมหานคร 10330
# 1800 019 448

# Apple World Trade Center
# 185 Greenwich Street
# New York, NY 10007
# (646) 802-3800

# Apple Marina Bay Sands
# 2 Bayfront Avenue, B2-06
# Singapore, 018972
# 1800 407 4949

# Apple Causeway Bay
# Hysan Place
# 500 Hennessy Road
# Causeway Bay
# 39793100

# Apple Brompton Road
# 17-27 Brompton Road
# London, SW1X 9LF
# 020 3089 3000

# Apple Alstertal
# Alstertal-Einkaufszentrum
# Poppenbüttel
# 22391 Hamburg
# (040) 6118315000

# Apple Akasya
# Akasya AVM
# Acıbadem Mah, Çeçen Sok No: 25
# Üsküdar İstanbul 34660
# (0216) 250 71 00

# Apple Dubai Mall
# The Dubai Mall
# Downtown
# Dubai
# 800 0444 9012

# Apple Highpoint
# Highpoint Shopping Centre
# 120-200 Rosamond Road
# Maribyrnong, VIC 3032
# (03) 9975 9700

# Apple Marché Saint-Germain
# 9 rue Clément
# 75006 Paris
# 01 85 65 48 00

# Apple Union Square
# 300 Post Street
# San Francisco, CA 94108
# (415) 486-4800

In [73]:
name = ['Apple Central World','Apple World Trade Center','Apple Marina Bay Sands',
        'Apple Causeway Bay','Apple Brompton Road','Apple Alstertal',
        'Apple Akasya','Apple Dubai Mall','Apple Highpoint',
        'Apple Marche Saint-Germain','Apple Union Square']

address = ['Pathumwan Bankgog 10330 1800 019 448',
           '185 Greenwich Street New York, NY 10007 (646) 802-3800',
           '2 Bayfront Avenue, B2-06 Singapore, 018972 1800 407 4949',
           'Hysan Place 500 Hennessy Road Causeway Bay 39793100',
           '17-27 Brompton Road London, SW1X 9LF 020 3089 3000',
           'Alstertal-Einkaufszentrum Poppenbuttel 22391 Hamburg (040) 6118315000',
           'Akasya AVM Acibadem Mah, Cecen Sok No: 25 Uskudar Istanbul 34660 (0216) 250 71 00',
           'The Dubai Mall Downtown Dubai 800 0444 9012',
           'Highpoint Shopping Centre 120-200 Rosamond Road Maribyrnong, VIC 3032 (03) 9975 9700',
           '9 rue Clement 75006 Paris 01 85 65 48 00',
           '300 Post Street San Francisco, CA 94108 (415) 486-4800']

store_number = list(range(1000,1000 + len(name)))

Physical_store = pd.DataFrame([],columns = ['store_number','address','name'])

Physical_store['store_number'] = pd.Series(store_number)
Physical_store['address'] = pd.Series(address)
Physical_store['name'] = pd.Series(name)

Physical_store

,store_number,address,name
0,1000,Pathumwan Bankgog 10330 1800 019 448,Apple Central World
1,1001,"185 Greenwich Street New York, NY 10007 (646) ...",Apple World Trade Center
2,1002,"2 Bayfront Avenue, B2-06 Singapore, 018972 180...",Apple Marina Bay Sands
3,1003,Hysan Place 500 Hennessy Road Causeway Bay 397...,Apple Causeway Bay
4,1004,"17-27 Brompton Road London, SW1X 9LF 020 3089 ...",Apple Brompton Road
5,1005,Alstertal-Einkaufszentrum Poppenbuttel 22391 H...,Apple Alstertal
6,1006,"Akasya AVM Acibadem Mah, Cecen Sok No: 25 Usku...",Apple Akasya
7,1007,The Dubai Mall Downtown Dubai 800 0444 9012,Apple Dubai Mall
8,1008,Highpoint Shopping Centre 120-200 Rosamond Roa...,Apple Highpoint
9,1009,9 rue Clement 75006 Paris 01 85 65 48 00,Apple Marche Saint-Germain


In [74]:
Physical_store.to_csv('Physical_store.csv') # saving data

# Prepare data for product line

In [75]:
product_line_dict = {'line_id':[],
                    'product_line_name':[],
                     'description':[]}

product_line_dict['product_line_name'] = ['Mac','iPad',
                                          'iPhone','Apple Watch',
                                          'Apple Vision Pro','AirPods',
                                          'TV & Home',
                                          'Accessories']

product_line_dict['description'] = ['If you can dream it,Mac can do it.',
                                    'Touch, draw, and type on one magical device.',
                                    'Designed to be loved.',
                                    'The ultimate device for a healthy life.',
                                    'Apple Vision Pro seamlessly blends digital content with your physical space. So you can work, watch, relive memories, and connect in ways never before possible. The era of spatial computing is here.',
                                    'The next evolution of sound and comfort.',
                                    'The future hits home. Simply connect your favorite devices and transform your house into a remarkably smart, convenient, and entertaining home. Elevate movie night with theater-like picture and sound. Play any song, in any room, from anywhere. And control lights, locks, and thermostats using Siri. All with the security and privacy of Apple.',
                                    'Accessorize in a snap. Find a MagSafe case, wallet, or charger that is right for you.']

product_line_dict['line_id'] = list(range(10,10 + len(product_line_dict['description'])))

product_line_df = pd.DataFrame(product_line_dict)
product_line_df.to_csv('Product_line.csv')


# Prepare data for Week days

In [76]:


def get_time_working_day():
  h_open = random.randint(7,11)
  h_close = random.randint(19,21)
  times = f'{h_open}:00:00',f'{h_close}:00:00'
  return times

def get_time_weekend():
  h_open = random.randint(10,12)
  h_close = random.randint(20,22)
  times = f'{h_open}:00:00',f'{h_close}:00:00'
  return times

day_name = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']
store_numbers = Physical_store['store_number'].values

week_day_dict = {'day_name':[],'opening_time':[],'closing_time':[],'store_number':[]}

for sn in store_numbers:
  ot,ct = get_time_working_day()
  for i in range(0,5): # for working day
    week_day_dict['store_number'].append(sn)
    week_day_dict['day_name'].append(day_name[i])
    week_day_dict['opening_time'].append(ot)
    week_day_dict['closing_time'].append(ct)

  week_day_dict['store_number'].append(sn)
  week_day_dict['day_name'].append(day_name[-1]) # for sunday
  week_day_dict['store_number'].append(sn)
  week_day_dict['day_name'].append(day_name[-2]) # for saturday
  ot,ct = get_time_weekend()
  week_day_dict['opening_time'].append(ot) # for sunday
  week_day_dict['closing_time'].append(ct) # for sunday
  ot,ct = get_time_weekend()
  week_day_dict['opening_time'].append(ot) # for saturday
  week_day_dict['closing_time'].append(ct) # for saturday

week_day_df = pd.DataFrame(week_day_dict)
week_day_df[:10]

,day_name,opening_time,closing_time,store_number
0,monday,11:00:00,20:00:00,1000
1,tuesday,11:00:00,20:00:00,1000
2,wednesday,11:00:00,20:00:00,1000
3,thursday,11:00:00,20:00:00,1000
4,friday,11:00:00,20:00:00,1000
5,sunday,12:00:00,22:00:00,1000
6,saturday,10:00:00,20:00:00,1000
7,monday,11:00:00,19:00:00,1001
8,tuesday,11:00:00,19:00:00,1001
9,wednesday,11:00:00,19:00:00,1001


In [77]:
week_day_df.to_csv('Week_day.csv')

# prepare mapping account to store

In [79]:
Account_df.head(5)

,Account_ID,Email,Phone_number,Login_verification_method,country,name,address,password
0,10000,Sara.Russ@gmail.com,541-525-9113,OTP,USA,Sara Russell,8779 Pine Rd 16501 Maryland,99dd3f4090d8fdc6500119f270ae60f6
1,10001,Beatrice.Pete@gmail.com,180-717-9888,Email,USA,Beatrice Peters,4783 Hickory Ln 50613 Massachusetts,27268e6ac894c1ed4ca1d8921f5a69a0
2,10002,Maisie.Cast@gmail.com,760-576-5640,device,USA,Maisie Castaneda,4894 Maple Rd 64075 California,f3c448081aab09091f4f5ed1eddf1762
3,10003,Beatrice.Rawl@gmail.com,546-369-351,device,USA,Beatrice Rawlings,3673 Cedar Blvd 27601 Vermont,33634c80f28bb8ed5190a3e4c006c170
4,10004,Steven.Russ@gmail.com,499-614-727,Email,USA,Steven Russell,9990 Oak Dr 58102 Nebraska,c04259ac5f289f13efa695db38f82149


In [80]:
mapping = {'Account_ID':[],'store_number':[]}
numstore_per_acc = np.random.randint(1,3,size = Account_df.shape[0])
for idx in range(0,Account_df.shape[0]):
  for sn in Physical_store['store_number'].sample(numstore_per_acc[idx]).values:
    mapping['Account_ID'].append(Account_df.loc[idx,'Account_ID'])
    mapping['store_number'].append(sn)
mapping_df = pd.DataFrame(mapping)
mapping_df[:10]

,Account_ID,store_number
0,10000,1007
1,10001,1005
2,10001,1007
3,10002,1002
4,10002,1008
5,10003,1010
6,10003,1002
7,10004,1006
8,10005,1008
9,10005,1003


In [82]:
mapping_df.to_csv('is_close_to.csv')

# Prepare data for product

### use pd to load the DataAppleStore - Apple_product.csv

In [87]:
product_df = pd.read_csv('DataAppleStore - Apple_product.csv')
product_df.drop(columns = ['Unnamed: 0'],inplace = True)
# installment_price contain null values
size_record = product_df.shape[0]
product_df['product_number'] = pd.Series(list(range(2000,2000 + size_record)))
lower_bound = abs(np.random.randn(size_record) * 20000 / product_df['price'].values).astype('int32') + 1
range_item = abs(np.random.randn(size_record) * 5000 / product_df['price'].values).astype('int32') + 1
upper_bound = lower_bound + range_item
final_num_item = []

for idx in range(0,size_record):
  final_num_item.append(random.randint(lower_bound[idx],upper_bound[idx]))
product_df['number_of_item_in_inventory'] = pd.Series(final_num_item)

product_line_map_id = {'Mac':10,'iPad' : 11, 'iPhone' : 12, 'Apple Watch' : 13,
                       'Apple Vision Pro' : 14,'AirPods' : 15, 'TV & Home' : 16,
                       'Accessories' : 17} # change Ipad to ipad interchangably and if error
line_id = []
for pln in product_df['product_line_name']:
  line_id.append(product_line_map_id[pln])
product_df['line_id'] = pd.Series(line_id)
product_df.drop(columns = ['product_line_name'],inplace = True)

product_df['description'] = product_df['description'].str.replace('‑', '-', regex=False)
product_df['description'] = product_df['description'].str.replace('/', 'or', regex=False)
product_df['description'] = product_df['description'].str.replace('"', '-inch', regex=False)
product_df['description'] = product_df['description'].str.replace('(', '', regex=False)
product_df['description'] = product_df['description'].str.replace(')', '', regex=False)

product_df['name'] = product_df['name'].str.replace('‑', '-', regex=False)
product_df['name'] = product_df['name'].str.replace('/', 'or', regex=False)
product_df['name'] = product_df['name'].str.replace('"', 'inch', regex=False)
product_df['name'] = product_df['name'].str.replace('(', '', regex=False)
product_df['name'] = product_df['name'].str.replace(')', '', regex=False)

product_df['Installment_price'] = product_df['price']/12


In [ ]:
product_df[20:23]

,description,product type,name,price,Installment_price,product_number,number_of_item_in_inventory,line_id
20,"6.9-inch Super Retina XDR display, A18 Pro chi...",Smartphone,iPhone 16 Pro Max,1199.0,99.916667,2020,5,12
21,"44mm or 40mm aluminum case, LTPO OLED Retina d...",Smartwatch,Apple Watch SE 2nd generation,249.0,20.750000,2021,32,13
22,"46mm or 42mm aluminum or titanium case, LTPO3 ...",Smartwatch,Apple Watch Series 10,399.0,33.250000,2022,56,13


In [88]:
product_df.to_csv('product.csv')

# gen purchase record and ongoing order



In [89]:
# Purchase record (record number, delivery option, confirm payment method,
#                  Account ID, store number) (Create Second)
# Ongoing order (order number, delivery option, unconfirmed payment method,
#                Account ID, store number)(Create Second)
# delivery option
# Purchase record
# must contain only 2 values which are “pick-up” and  “standard-delivery”
# confirm payment method
# Purchase record
# must contain only 2 values which are “ATM”, and “credit card”
# unconfirm payment method
# Ongoing order
# must contain only 2 values which are “pick-up” and  “standard-delivery”



In [90]:
product_df['product_number'].values

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032,
       2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043,
       2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054,
       2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065,
       2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073])

In [91]:
Account_df['Account_ID'].values

array([10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008,
       10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017,
       10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026,
       10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035,
       10036, 10037, 10038, 10039, 10040, 10041, 10042, 10043, 10044,
       10045, 10046, 10047, 10048, 10049, 10050, 10051, 10052, 10053,
       10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062,
       10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071,
       10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080,
       10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089,
       10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098,
       10099, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107,
       10108, 10109, 10110, 10111, 10112, 10113, 10114, 10115, 10116,
       10117, 10118, 10119, 10120, 10121, 10122, 10123, 10124, 10125,
       10126, 10127,

In [92]:
Physical_store['store_number'].values

array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010])

In [93]:
delivery_option = ['pick-up','standard-delivery']
payment_method = ['ATM','credit card']

sample_size = 1000
account_size = Account_df.shape[0]
store_size = Physical_store.shape[0]

Purchase_record = {'record_number':[],
                   'delivery_option':[],
                   'confirm_payment_method':[],
                   'Account_ID':[],
                   'store_number':[]}

Ongoing_order = {  'order_number':[],
                   'delivery_option':[],
                   'unconfirmed_payment_method':[],
                   'Account_ID':[],
                   'store_number':[]}

base_id = 100
for idx in range(0,sample_size):
  Purchase_record['record_number'].append(base_id + idx)
  Purchase_record['delivery_option'].append(delivery_option[random.randint(0,1)])
  Purchase_record['confirm_payment_method'].append(payment_method[random.randint(0,1)])
  Purchase_record['Account_ID'].append(Account_df['Account_ID'].loc[random.randint(0,account_size - 1)])
  Purchase_record['store_number'].append(Physical_store['store_number'].loc[random.randint(0,store_size - 1)])

base_id = 400
for idx in range(0,sample_size):
  Ongoing_order['order_number'].append(base_id + idx)
  Ongoing_order['delivery_option'].append(delivery_option[random.randint(0,1)])
  Ongoing_order['unconfirmed_payment_method'].append(payment_method[random.randint(0,1)])
  Ongoing_order['Account_ID'].append(Account_df['Account_ID'].loc[random.randint(0,account_size - 1)])
  Ongoing_order['store_number'].append(Physical_store['store_number'].loc[random.randint(0,store_size - 1)])

Purchase_record_df = pd.DataFrame(Purchase_record)
Ongoing_order_df = pd.DataFrame(Ongoing_order)

In [94]:
Purchase_record_df.to_csv('Purchase_record.csv')
Ongoing_order_df.to_csv('Ongoing_order.csv')

# include and consist of table

In [95]:
print(product_df.shape[0])
print(Purchase_record_df.shape[0])
print(Ongoing_order_df.shape[0])

74
1000
1000


In [98]:
include_mapping = {'record_number':[],'product_number':[],'unit_ordered':[]}
consist_of_mapping = {'order_number':[],'product_number':[],'unit_ordered':[]}


for idx in range(0,Purchase_record_df.shape[0]):
  sample_indices = product_df.sample(random.randint(1,random.randint(1,3))).index
  for sp_idx in sample_indices:
    include_mapping['record_number'].append(Purchase_record_df.loc[idx,'record_number'])
    include_mapping['product_number'].append(product_df.loc[sp_idx,'product_number'])
    max_num_ordered = (product_df.loc[sp_idx,'number_of_item_in_inventory'] // 10) + 1
    include_mapping['unit_ordered'].append(random.randint(1,max_num_ordered))

for idx in range(0,Ongoing_order_df.shape[0]):
  sample_indices = product_df.sample(random.randint(1,random.randint(1,3))).index
  for sp_idx in sample_indices:
    consist_of_mapping['order_number'].append(Ongoing_order_df.loc[idx,'order_number'])
    consist_of_mapping['product_number'].append(product_df.loc[sp_idx,'product_number'])
    max_num_ordered = (product_df.loc[sp_idx,'number_of_item_in_inventory'] // 10) + 1
    consist_of_mapping['unit_ordered'].append(random.randint(1,max_num_ordered))

include_df = pd.DataFrame(include_mapping)
consist_of_df = pd.DataFrame(consist_of_mapping)
print(f'final len of include df is {include_df.shape[0]}')
print(f'final len of consist_of df is {consist_of_df.shape[0]}')
include_df.to_csv('include.csv')
consist_of_df.to_csv('consist_of.csv')

final len of include df is 1496
final len of consist_of df is 1460


# product compatibility

In [99]:
non_accessory_product_df = product_df[product_df['line_id'] != 17].reset_index(drop=True)
accessory_product_df = product_df[product_df['line_id'] == 17].reset_index(drop=True)
compat_mapping = {'product_number':[],'accessory_number':[]}
for idx in range(non_accessory_product_df.shape[0]):
  if((random.randint(0,2) % 2) == 1):
    continue; # 2 out of 3 product will get accessory compatibility
  sample_indices = accessory_product_df.sample(random.randint(1,4)).index
  for sp_idx in sample_indices:
    compat_mapping['product_number'].append(non_accessory_product_df.loc[idx,'product_number'])
    compat_mapping['accessory_number'].append(accessory_product_df.loc[sp_idx,'product_number'])

compat_df = pd.DataFrame(compat_mapping)
print(f' final compatability len = {compat_df.shape[0]}')
compat_df.to_csv('compatability.csv')

 final compatability len = 50


# product type

In [103]:
product_df['product type'].unique()

array(['Laptop', 'Desktop', 'Tablet', 'Smartphone', 'Smartwatch', 'VR',
       'Earbuds', 'Over-ear Headphones', 'Streaming Device',
       'Smart Speaker', 'iPhone Case', 'iPad Case', 'iPad Keyboard',
       'Mac Case', 'Apple Watch Band'], dtype=object)

In [105]:
product_df['line_id'].unique()

array([10, 11, 12, 13, 14, 15, 16, 17])

In [110]:
type_list = list(product_df['product type'].values)
line_id_list = list(product_df['line_id'].values)

In [122]:
type_id_dict = {}
for idx in range(0,len(type_list)):
  type_id_dict[type_list[idx]] = int(line_id_list[idx])

product_type_list = []
line_id_list = [] # reused name
for t in product_df['product type'].unique():
  product_type_list.append(t)
  line_id_list.append(type_id_dict[t])

In [123]:
product_type_df = pd.DataFrame({'type':product_type_list,'line_id':line_id_list})

In [124]:
product_type_df

,type,line_id
0,Laptop,10
1,Desktop,10
2,Tablet,11
3,Smartphone,12
4,Smartwatch,13
5,VR,14
6,Earbuds,15
7,Over-ear Headphones,15
8,Streaming Device,16
9,Smart Speaker,16


In [126]:
product_type_df.to_csv('product_type.csv')

In [129]:
!mkdir gen_file
!mv Apple_account.csv gen_file/
!mv Ongoing_order.csv gen_file/
!mv Physical_store.csv gen_file/
!mv Product_line.csv gen_file/
!mv Purchase_record.csv gen_file/
!mv Week_day.csv gen_file/
!mv compatability.csv gen_file/
!mv consist_of.csv gen_file/
!mv include.csv gen_file/
!mv is_close_to.csv gen_file/
!mv product_type.csv gen_file/


mkdir: cannot create directory ‘gen_file’: File exists
mv: cannot stat 'Apple_account.csv': No such file or directory
mv: cannot stat 'Ongoing_order.csv': No such file or directory
mv: cannot stat 'Physical_store.csv': No such file or directory
mv: cannot stat 'Product_line.csv': No such file or directory
mv: cannot stat 'Purchase_record.csv': No such file or directory
mv: cannot stat 'Week_day.csv': No such file or directory
mv: cannot stat 'compatability.csv': No such file or directory
mv: cannot stat 'consist_of.csv': No such file or directory
mv: cannot stat 'include.csv': No such file or directory
mv: cannot stat 'is_close_to.csv': No such file or directory


In [131]:
!zip -r gen_file.zip gen_file

  adding: gen_file/ (stored 0%)
  adding: gen_file/compatability.csv (deflated 60%)
  adding: gen_file/Week_day.csv (deflated 83%)
  adding: gen_file/Physical_store.csv (deflated 37%)
  adding: gen_file/consist_of.csv (deflated 60%)
  adding: gen_file/include.csv (deflated 60%)
  adding: gen_file/Apple_account.csv (deflated 55%)
  adding: gen_file/Purchase_record.csv (deflated 79%)
  adding: gen_file/Product_line.csv (deflated 41%)
  adding: gen_file/is_close_to.csv (deflated 68%)
  adding: gen_file/Ongoing_order.csv (deflated 78%)
  adding: gen_file/product_type.csv (deflated 29%)
